#  LangChain으로 데이터 생성

LLM으로 실제 데이터를 처리하다 보면 데이터가 너무 적거나 다양한 상황을 고려하지 못하는 경우    
파인 튜닝이나 LLM 성능 평가가 어려워지게 됨

In [1]:
!pip install langchain langchain-openai pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.8 MB/s eta 0:00:00


In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [3]:
from google.colab import userdata
key = userdata.get('Openai')

In [7]:
import pandas as pd
df_VOC = pd.read_csv('/content/drive/MyDrive/VOC_Data.csv')
df_VOC

,VOC,성별,연령,처리부서,평가점수
0,챗봇이 답변을 이해하지 못해 상담원 연결이 지연되었습니다. 심야 시간대라 급한 문의...,남성,30대,고객지원팀,2
1,챗봇을 통해 은행 업무를 처리하는데 간편하고 빠르게 진행할 수 있어 만족했습니다. ...,여성,20대,서비스개발팀,5
2,사용 중 챗봇이 오류가 나서 중간에 멈췄습니다. 재로그인 후 다시 시도해야 했습니다...,남성,40대,기술지원팀,3
3,정보를 찾는 데 있어 챗봇이 유용하게 작동했습니다. 특히 예약 관련 정보를 쉽게 찾...,여성,50대,사용자경험팀,4
4,친절하고 빠른 응대로 불편함 없이 문제를 해결할 수 있었습니다. 처음에는 걱정했지만...,여성,60대,고객만족팀,5


In [8]:
sample_VOC = df_VOC.sample(3)
sample_VOC

,VOC,성별,연령,처리부서,평가점수
2,사용 중 챗봇이 오류가 나서 중간에 멈췄습니다. 재로그인 후 다시 시도해야 했습니다...,남성,40대,기술지원팀,3
0,챗봇이 답변을 이해하지 못해 상담원 연결이 지연되었습니다. 심야 시간대라 급한 문의...,남성,30대,고객지원팀,2
4,친절하고 빠른 응대로 불편함 없이 문제를 해결할 수 있었습니다. 처음에는 걱정했지만...,여성,60대,고객만족팀,5


In [9]:
# 딕셔너리화
sample_VOC.to_dict(orient='records')

[{'VOC': '사용 중 챗봇이 오류가 나서 중간에 멈췄습니다. 재로그인 후 다시 시도해야 했습니다. 이러한 점이 개선되길 바랍니다.',
  '성별': '남성',
  '연령': '40대',
  '처리부서': '기술지원팀',
  '평가점수': 3},
 {'VOC': '챗봇이 답변을 이해하지 못해 상담원 연결이 지연되었습니다. 심야 시간대라 급한 문의였는데 답답했습니다. 빠른 답변과 정확성을 개선해주세요.',
  '성별': '남성',
  '연령': '30대',
  '처리부서': '고객지원팀',
  '평가점수': 2},
 {'VOC': '친절하고 빠른 응대로 불편함 없이 문제를 해결할 수 있었습니다. 처음에는 걱정했지만 좋은 경험이었습니다.',
  '성별': '여성',
  '연령': '60대',
  '처리부서': '고객만족팀',
  '평가점수': 5}]

In [17]:
# 하나의 딕셔너리로 만들기
sample_VOC_str = sample_VOC.to_dict(orient='records')
str(sample_VOC_str)

"[{'VOC': '사용 중 챗봇이 오류가 나서 중간에 멈췄습니다. 재로그인 후 다시 시도해야 했습니다. 이러한 점이 개선되길 바랍니다.', '성별': '남성', '연령': '40대', '처리부서': '기술지원팀', '평가점수': 3}, {'VOC': '챗봇이 답변을 이해하지 못해 상담원 연결이 지연되었습니다. 심야 시간대라 급한 문의였는데 답답했습니다. 빠른 답변과 정확성을 개선해주세요.', '성별': '남성', '연령': '30대', '처리부서': '고객지원팀', '평가점수': 2}, {'VOC': '친절하고 빠른 응대로 불편함 없이 문제를 해결할 수 있었습니다. 처음에는 걱정했지만 좋은 경험이었습니다.', '성별': '여성', '연령': '60대', '처리부서': '고객만족팀', '평가점수': 5}]"

In [11]:
example_dict = [ {'example': str(voc_str)[1:-1].replace("'",'')} for voc_str in sample_VOC_str]
example_dict

[{'example': 'VOC: 사용 중 챗봇이 오류가 나서 중간에 멈췄습니다. 재로그인 후 다시 시도해야 했습니다. 이러한 점이 개선되길 바랍니다., 성별: 남성, 연령: 40대, 처리부서: 기술지원팀, 평가점수: 3'},
 {'example': 'VOC: 챗봇이 답변을 이해하지 못해 상담원 연결이 지연되었습니다. 심야 시간대라 급한 문의였는데 답답했습니다. 빠른 답변과 정확성을 개선해주세요., 성별: 남성, 연령: 30대, 처리부서: 고객지원팀, 평가점수: 2'},
 {'example': 'VOC: 친절하고 빠른 응대로 불편함 없이 문제를 해결할 수 있었습니다. 처음에는 걱정했지만 좋은 경험이었습니다., 성별: 여성, 연령: 60대, 처리부서: 고객만족팀, 평가점수: 5'}]

In [18]:
PREFIX = '{subject}에 대한 데이터를 생성하세요. 예시는 다음과 같습니다.:'
PREFIX

'{subject}에 대한 데이터를 생성하세요. 예시는 다음과 같습니다.:'

In [19]:
SUFFIX = '''데이터를 생성할 때, 다음의 내용을 고려하세요.
{extra}'''
SUFFIX

'데이터를 생성할 때, 다음의 내용을 고려하세요.\n{extra}'

### Prefix, Suffix, Examples와 example_prompt를 이용해 최종 프롬프트를 생성

In [22]:
example_dict

[{'example': 'VOC: 사용 중 챗봇이 오류가 나서 중간에 멈췄습니다. 재로그인 후 다시 시도해야 했습니다. 이러한 점이 개선되길 바랍니다., 성별: 남성, 연령: 40대, 처리부서: 기술지원팀, 평가점수: 3'},
 {'example': 'VOC: 챗봇이 답변을 이해하지 못해 상담원 연결이 지연되었습니다. 심야 시간대라 급한 문의였는데 답답했습니다. 빠른 답변과 정확성을 개선해주세요., 성별: 남성, 연령: 30대, 처리부서: 고객지원팀, 평가점수: 2'},
 {'example': 'VOC: 친절하고 빠른 응대로 불편함 없이 문제를 해결할 수 있었습니다. 처음에는 걱정했지만 좋은 경험이었습니다., 성별: 여성, 연령: 60대, 처리부서: 고객만족팀, 평가점수: 5'}]

In [23]:
Example_Template = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=PREFIX,
    examples=example_dict,
    suffix=SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=Example_Template,
)

In [24]:
print(prompt_template.format(subject='챗봇 회사의 VOC',
                             extra='기존의 예시를 그대로 활용하지 말고, 자세한 내용으로 기재'))

챗봇 회사의 VOC에 대한 데이터를 생성하세요. 예시는 다음과 같습니다.:

VOC: 사용 중 챗봇이 오류가 나서 중간에 멈췄습니다. 재로그인 후 다시 시도해야 했습니다. 이러한 점이 개선되길 바랍니다., 성별: 남성, 연령: 40대, 처리부서: 기술지원팀, 평가점수: 3

VOC: 챗봇이 답변을 이해하지 못해 상담원 연결이 지연되었습니다. 심야 시간대라 급한 문의였는데 답답했습니다. 빠른 답변과 정확성을 개선해주세요., 성별: 남성, 연령: 30대, 처리부서: 고객지원팀, 평가점수: 2

VOC: 친절하고 빠른 응대로 불편함 없이 문제를 해결할 수 있었습니다. 처음에는 걱정했지만 좋은 경험이었습니다., 성별: 여성, 연령: 60대, 처리부서: 고객만족팀, 평가점수: 5

데이터를 생성할 때, 다음의 내용을 고려하세요.
기존의 예시를 그대로 활용하지 말고, 자세한 내용으로 기재


Generator

스키마를 정하고 Temperature와 같은 파라미터를 설정


In [26]:
llm = ChatOpenAI(api_key=key, model='gpt-4o', temperature=0.2, max_tokens=2048)

generate_chain = prompt_template | llm | StrOutputParser()
gen_result = generate_chain.invoke({'subject':'챗봇 회사의 VOC',
                                    'extra':'''기존의 예시를 그대로 활용하지 말고,
                                    자세한 내용으로 기재'''})
print(gen_result)

VOC: 챗봇이 특정 질문에 대해 반복적으로 같은 답변만 제공하여 원하는 정보를 얻지 못했습니다. 결국 상담원과의 연결이 필요했으며, 이 과정에서 시간이 많이 소요되었습니다. 챗봇의 답변 다양성과 정확성을 높여주셨으면 합니다., 성별: 여성, 연령: 50대, 처리부서: 기술지원팀, 평가점수: 2

VOC: 챗봇과의 대화가 매끄럽고 자연스러워서 마치 사람과 대화하는 것 같았습니다. 문제 해결도 신속하게 이루어졌고, 매우 만족스러웠습니다. 앞으로도 이런 서비스를 기대합니다., 성별: 남성, 연령: 20대, 처리부서: 고객만족팀, 평가점수: 5

VOC: 챗봇이 제공하는 정보가 최신이 아니어서 혼란을 겪었습니다. 업데이트가 필요해 보이며, 특히 제품 관련 정보는 실시간으로 반영되었으면 좋겠습니다. 이러한 부분이 개선된다면 더 나은 서비스가 될 것 같습니다., 성별: 여성, 연령: 30대, 처리부서: 정보관리팀, 평가점수: 3

VOC: 심야 시간에 챗봇을 사용했는데, 예상보다 빠르게 문제를 해결할 수 있었습니다. 다만, 일부 기술적인 질문에 대한 답변이 부족하여 추가적인 검색이 필요했습니다. 기술 관련 데이터베이스를 확충해주시면 좋겠습니다., 성별: 남성, 연령: 40대, 처리부서: 기술지원팀, 평가점수: 4

VOC: 챗봇이 제공하는 서비스가 전반적으로 만족스러웠으나, 특정 기능에 대한 설명이 부족하여 사용에 어려움을 겪었습니다. 기능 설명을 보다 상세하게 제공해주시면 좋겠습니다., 성별: 여성, 연령: 60대, 처리부서: 고객지원팀, 평가점수: 3


gen_result를 넣고 이를 요약하는 보고서를 작성

In [28]:
VOC_system_prompt = '''챗봇을 이용하는 사용자의 VOC 정보가 아래에 주어집니다.
해당 내용을 요약하여 설명하는 보고서를 작성하세요.
보고서는 -로 구분된 리스트 형식으로, 최소 6문장에서 최대 8문장으로 작성하세요.
보고서에는 VOC의 내용과 관리자의 입장에서 이를 바라본 시각이 들어가야 합니다.
순서대로 나열하듯 설명하지 말고, 전반적인 관점에서 전체 내용을 설명하세요.'''

In [29]:
prompt_template  = ChatPromptTemplate.from_messages([
    ('system', VOC_system_prompt),
    ('user', '{VOC}')
])
llm = ChatOpenAI(api_key= key, model='gpt-4o')

summarize_chain = prompt_template | llm | StrOutputParser()
summarize_result = summarize_chain.invoke({'VOC':gen_result})

print(summarize_result)

- 이번 VOC 분석에서는 챗봇 서비스에 대한 다양한 사용자 경험이 반영되었습니다. 챗봇의 답변이 반복적이거나 정보의 업데이트가 미흡한 점을 지적한 고객들이 있었으며, 이는 특히 기술지원팀과 정보관리팀의 중점을 두어야 할 부분입니다.
- 50대 여성 사용자는 챗봇의 답변 일관성 문제로 인해 대화 흐름이 원활하지 않았고, 추가적인 시간 소요가 발생했다고 평가했습니다. 이 사례는 챗봇 AI의 응답 다양성과 정확성 향상이 중요함을 시사합니다.
- 반면, 20대 남성 사용자는 챗봇과의 자연스러운 대화와 신속한 문제 해결을 긍정적으로 평가했습니다. 이는 챗봇의 언어 처리가 일부 사용자에게는 충분히 만족스럽게 작동하고 있음을 보여줍니다.
- 챗봇의 정보가 최신화되지 않아서 30대 사용자가 혼란을 겪은 경험은 제품 정보 데이터베이스의 실시간 갱신 필요성을 부각시켰습니다. 이는 정보관리팀의 우선 과제로 인식되어야 합니다.
- 또한, 심야 시간에도 기대 이상의 문제 해결 속도를 경험한 40대 사용자의 긍정적인 피드백은 챗봇의 운영 시간을 확장하는 데 대한 가능성을 보여줍니다.
- 마지막으로, 60대 사용자는 챗봇 기능 설명 부족을 지적하며 세부 가이드의 필요성을 언급했습니다. 고객지원팀은 사용 설명서의 개선을 통해 전반적인 사용 편의성을 높여야 할 것입니다.
- 전반적으로 챗봇의 대화 방식과 정보의 정확성을 강화하고, 모든 연령대의 사용자가 손쉽게 활용할 수 있는 기능 개선이 필요합니다. 이와 같은 시스템 향상을 통해 고객 만족도를 높일 수 있을 것입니다.


In [30]:
full_data = f'''
VOC Data
{gen_result}
---
VOC Summary
{summarize_result}
'''
print(full_data)



VOC Data
VOC: 챗봇이 특정 질문에 대해 반복적으로 같은 답변만 제공하여 원하는 정보를 얻지 못했습니다. 결국 상담원과의 연결이 필요했으며, 이 과정에서 시간이 많이 소요되었습니다. 챗봇의 답변 다양성과 정확성을 높여주셨으면 합니다., 성별: 여성, 연령: 50대, 처리부서: 기술지원팀, 평가점수: 2

VOC: 챗봇과의 대화가 매끄럽고 자연스러워서 마치 사람과 대화하는 것 같았습니다. 문제 해결도 신속하게 이루어졌고, 매우 만족스러웠습니다. 앞으로도 이런 서비스를 기대합니다., 성별: 남성, 연령: 20대, 처리부서: 고객만족팀, 평가점수: 5

VOC: 챗봇이 제공하는 정보가 최신이 아니어서 혼란을 겪었습니다. 업데이트가 필요해 보이며, 특히 제품 관련 정보는 실시간으로 반영되었으면 좋겠습니다. 이러한 부분이 개선된다면 더 나은 서비스가 될 것 같습니다., 성별: 여성, 연령: 30대, 처리부서: 정보관리팀, 평가점수: 3

VOC: 심야 시간에 챗봇을 사용했는데, 예상보다 빠르게 문제를 해결할 수 있었습니다. 다만, 일부 기술적인 질문에 대한 답변이 부족하여 추가적인 검색이 필요했습니다. 기술 관련 데이터베이스를 확충해주시면 좋겠습니다., 성별: 남성, 연령: 40대, 처리부서: 기술지원팀, 평가점수: 4

VOC: 챗봇이 제공하는 서비스가 전반적으로 만족스러웠으나, 특정 기능에 대한 설명이 부족하여 사용에 어려움을 겪었습니다. 기능 설명을 보다 상세하게 제공해주시면 좋겠습니다., 성별: 여성, 연령: 60대, 처리부서: 고객지원팀, 평가점수: 3
---
VOC Summary
- 이번 VOC 분석에서는 챗봇 서비스에 대한 다양한 사용자 경험이 반영되었습니다. 챗봇의 답변이 반복적이거나 정보의 업데이트가 미흡한 점을 지적한 고객들이 있었으며, 이는 특히 기술지원팀과 정보관리팀의 중점을 두어야 할 부분입니다.
- 50대 여성 사용자는 챗봇의 답변 일관성 문제로 인해 대화 흐름이 원활하지 않았고, 추가적인 시간 소요가 발생했다고 평가했습니다. 이 사

## RAG 데이터 만들기

In [42]:
system_prompt = '''아래에 텍스트가 주어집니다. 텍스트의 내용을 참고하여, 질문과 답변의 형식 데이터를 3개 생성하세요.
출력 규칙은 다음과 같습니다.

Question: 텍스트에서 주요 사항에 대한 질문을 작성하세요. 너무 쉽거나 지엽적인 질문은 만들지 마세요. 항상 의문문으로 만드세요.
Answer: 답변은 "그것도 몰라?" 로 항상 시작해야 하며, 답변을 하기 전
답변과 관련 있는 텍스트의 내용을 그대로 인용하여
"[근거]를 보면 알 수 있잖아"와 같은 형태로 출력하고,
그 이후 답변을 작성하세요.
근거를 제외한 답변은 모두 건방진 반말로 쓰세요.
---
예시:
Question: 진화가 일어나는 주요 작동 기제는 무엇인가요?
Answer: 그것도 몰라? "진화가 일어나는 주요 작동 기제는,
생물 집단과 환경의 상호 관계에 의해 유전형질이 선택되는 자연선택과,
집단 안에서 이루어지는 유전자 부동이다." 근거를 보면 알 수 있잖아.
자연선택이랑 유전자 부동이 진화의 핵심 메커니즘이야.
'''

In [43]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        ('user', '{context}')
    ]
)

llm = ChatOpenAI(api_key= key,temperature=0.5, model='gpt-4o', max_tokens=2048)

chain = prompt | llm | StrOutputParser()

In [44]:
context = '''야구의 명칭의 유래는 1871년 미국에서 일본으로 방문하여 일본 제일고등중학교[6]의 외국인 교사로 활동하는 호레이스 윌슨이 'baseball'이라는 운동 종목을 학생들에게 가르쳤고, 1894년 제일고등중학교에서 재학 중이던 주만 가나에가 'baseball'을 일본어 '야구'(일본어: 野球, やきゅう 야큐[*])로 번역하였고 이 말이 한반도로 들어와 사용되고 있는 것이다. 이에 앞서 4년 전인 1890년 일본의 문학가 마사오카 시키가 '배터(batter)', '러너(runner)', '베이스 온 볼스(base on balls)', '스트레이트(straight)', '플라이 볼(fly ball)', '쇼트스톱(shortstop)' 등의 미국식 야구 용어를 '타자', '주자', '사구 (四球, 대한민국에선 사구 대신 볼넷으로 사용)', '직구'(대한민국에서는 포심 패스트볼, 속구, 컷 패스트볼로 또 다르게 부름), '플라이'(대한민국에서는 뜬공으로도 부름), '단차'(短遮, 이후에는 주만 가나에의 교육에 따라 유격수로 번역함)로 번역하였다.[7] 때때로 야구를 비슷한 경기인 소프트볼(softball)에 대비하여 하드볼(hardball)로 부르기도 한다.
야구의 기원은 확실하게 짚어내기가 힘들다. 1344년에 출간된 프랑스의 한 책에 성직자들이 야구와 흡사한 라 술(La soule)이라는 게임을 즐기는 모습을 담은 삽화가 실려있다.[8] 원래 야구는 미국의 애브너 더블데이라는 군인 출신의 인물이 1839년에 쿠퍼스타운에서 창안했다고 알려졌다. 하지만 그가 직접 야구를 창안했다는 구체적인 증거가 없으며, 광산 기술자 출신의 애브너 그레이브스의 증언이 있었으나 별로 신뢰하기 어렵기 때문에 야구는 유럽에서 미국으로 전해진 것으로 추측된다.
방망이와 공을 이용해 하는 경기는 영국에서 여러 가지 종류로 발전하는데, 영국 동남부에서는 크리켓으로, 남서부에서는 원홀 캣(또는 원 오캣)의 이름으로 발전했으며, 야구와 흡사한 경기인 라운더스도 있었다. 이런 종류의 경기들은 아메리카 대륙으로 옮겨와 두가지 형태로 진화했는데, 뉴욕 주변에서 행해진 타운볼은 각각 9명의 선수로 이루어진 두 팀이 하는 경기로 정비되었다.'''

In [45]:
example = chain.invoke({'context':context})
print(example)

Question: 야구라는 명칭은 어떻게 유래되었나요?
Answer: 그것도 몰라? "1871년 미국에서 일본으로 방문하여 일본 제일고등중학교의 외국인 교사로 활동하는 호레이스 윌슨이 'baseball'이라는 운동 종목을 학생들에게 가르쳤고, 1894년 제일고등중학교에서 재학 중이던 주만 가나에가 'baseball'을 일본어 '야구'로 번역하였고 이 말이 한반도로 들어와 사용되고 있는 것이다." 근거를 보면 알 수 있잖아. 호레이스 윌슨이 일본에 야구를 가르치고, 주만 가나에가 그걸 '야구'로 번역한 거야.

Question: 야구의 기원에 대해 어떤 추측이 있나요?
Answer: 그것도 몰라? "야구의 기원은 확실하게 짚어내기가 힘들다. 1344년에 출간된 프랑스의 한 책에 성직자들이 야구와 흡사한 라 술이라는 게임을 즐기는 모습을 담은 삽화가 실려있다. 원래 야구는 미국의 애브너 더블데이라는 군인 출신의 인물이 1839년에 쿠퍼스타운에서 창안했다고 알려졌다. 하지만 그가 직접 야구를 창안했다는 구체적인 증거가 없으며, 광산 기술자 출신의 애브너 그레이브스의 증언이 있었으나 별로 신뢰하기 어렵기 때문에 야구는 유럽에서 미국으로 전해진 것으로 추측된다." 근거를 보면 알 수 있잖아. 야구는 유럽에서 미국으로 전해졌을 가능성이 크다고 봐야지.

Question: 야구가 영국에서 어떤 형태로 발전했나요?
Answer: 그것도 몰라? "방망이와 공을 이용해 하는 경기는 영국에서 여러 가지 종류로 발전하는데, 영국 동남부에서는 크리켓으로, 남서부에서는 원홀 캣(또는 원 오캣)의 이름으로 발전했으며, 야구와 흡사한 경기인 라운더스도 있었다." 근거를 보면 알 수 있잖아. 영국에서 크리켓, 원홀 캣, 라운더스 같은 걸로 발전했지.
